# PROYECTO DE ACCIDENTALIDAD. PROVINCIA DE VALENCIA

## EDA y limpieza de datos. Creación de dataframes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
import os

from functions import *

In [2]:
data_path = r"C:\Users\alejandro.gc\Desktop\Proyectos\Accidentalidad\Accidentes_VALENCIA\data\2019 - 2024 (a 31 agosto).xlsx"

In [4]:
hojas = ["Accidentes", "Características Vía", "Características Tramo", "Personas Implicadas", "Vehículos"]
dfs = {}
for hoja in hojas:
    dfs[hoja] = pd.read_excel(data_path, sheet_name=hoja)

df_accidentes = dfs["Accidentes"]
df_vias = dfs["Características Vía"]
df_tramos = dfs["Características Tramo"]
df_personas = dfs["Personas Implicadas"]
df_vehiculos = dfs["Vehículos"]

In [5]:
#Estandarizamos los header para trabajar más comodo

lista_df = [df_accidentes, df_vias, df_tramos, df_personas, df_vehiculos]

for df in lista_df:
    estandarizar_columnas(df);

In [6]:
# Se identifica el número de registros de cada uno de los datframes generados 

nombres_df = ["df_accidentes", "df_vias", "df_tramos", "df_personas", "df_vehiculos"]

valuecounts = []

for nombre, df in zip(nombres_df, lista_df):
    suma = df["codigo del accidente"].value_counts().sum()
    valuecounts.append((nombre, suma))

for nombre, suma in valuecounts:
    print(f"{nombre} : {suma}")

df_accidentes : 4138
df_vias : 4138
df_tramos : 4251
df_personas : 7656
df_vehiculos : 5763


### Dataframe de accidentes

In [7]:
df_accidentes.columns

Index(['codigo del accidente', 'fecha inicio exportado', 'fecha fin exportado',
       'total personas', 'total victimas', 'victimas mortales',
       'victimas graves', 'victimas leves', 'ilesos', 'se desconoce',
       'titularidad de la via', 'fecha del accidente',
       'ultima accion realizada', 'no contabilizable', 'agente', 'unidad',
       'fecha de modificacion en arena', 'longitud', 'latitud', 'hacia',
       'tramo de red asignado', 'carretera en informe', 'pk en informe',
       'carretera de trabajo', 'pk de trabajo', 'sentido', 'zona', 'municipio',
       'poblacion', 'factores concurrentes', 'tipo de via', 'tipo de nudo',
       'informacion de nudo', 'tipo de enlace', 'cruza con',
       'regulacion prioridad', 'tipo de accidente', 'salida de via', 'suceso',
       'sentido contrario', 'viento fuerte', 'estado meteorologico',
       'iluminacion', 'superficie firme', 'animal salvaje', 'animal domestico',
       'visibilidad restringida por', 'tipo de dia', 'nivel de ci

In [8]:
delete_acc_columns = ['fecha inicio exportado', 'fecha fin exportado', 'titularidad de la via',
                      'ultima accion realizada', 'no contabilizable', 'agente', 'unidad',
                       'fecha de modificacion en arena', 'hacia', 'tramo de red asignado',
                       'pk en informe', 'carretera de trabajo', 'pk de trabajo', 'poblacion',
                       'tipo de enlace', 'cruza con', 'regulacion prioridad', 'animal salvaje',
                       'animal domestico', 'visibilidad restringida por', 'nivel de circulacion',
                       'observaciones del accidente', 'observaciones del tecnico', 'niebla']

drop_columns_df(df_accidentes, delete_acc_columns);

In [9]:
# Añadimos las columnas de hora, dia semana y mes en el que se produjo el accidente.
df_accidentes["hora"] = df_accidentes['fecha del accidente'].dt.hour
df_accidentes["mes"] = df_accidentes['fecha del accidente'].dt.month
df_accidentes["dia semana"] = df_accidentes['fecha del accidente'].dt.day_name()
df_accidentes["dia semana orden"] = df_accidentes['fecha del accidente'].dt.dayofweek + 1 #Pandas define el lunes como 0 por defecto

In [10]:
# Los valores nulos de latitud, longitud y carretera en informe se van a mentener por ahora.
# A continuación, se corrigen los nulos de las columnas "tipo de nudo" e "informacion de nudo"

df_accidentes["tipo de nudo"] = df_accidentes["tipo de nudo"].fillna("Fuera de nudo o de su zona de influencia (>20m / >200m)")
df_accidentes["informacion de nudo"] =  df_accidentes["informacion de nudo"].fillna("Fuera de nudo")

# Rellenamos con NO los valores nulos de la columna "viento fuerte"
df_accidentes["viento fuerte"] = df_accidentes["viento fuerte"].fillna("NO")

# Rellenamos con "se desconoce" los valores nulos de la columna "estado meteorologico" e "iluminacion"
df_accidentes["estado meteorologico"] = df_accidentes["estado meteorologico"].fillna("Se desconoce")
df_accidentes["iluminacion"] = df_accidentes["iluminacion"].fillna("Se desconoce")
df_accidentes["carretera en informe"] = df_accidentes["carretera en informe"].fillna("Se desconoce")

In [11]:
df_accidentes.isnull().sum()

codigo del accidente           0
total personas                 0
total victimas                 0
victimas mortales              0
victimas graves                0
victimas leves                 0
ilesos                         0
se desconoce                   0
fecha del accidente            0
longitud                     140
latitud                      142
carretera en informe           0
sentido                        0
zona                           0
municipio                      0
factores concurrentes          0
tipo de via                    0
tipo de nudo                   0
informacion de nudo            0
tipo de accidente              0
salida de via                  0
suceso                         0
sentido contrario              0
viento fuerte                  0
estado meteorologico           0
iluminacion                    0
superficie firme               0
tipo de dia                    0
descripcion del accidente      0
hora                           0
mes       

### Dataframe de características de vías

In [12]:
df_vias.columns

Index(['codigo del accidente', 'numero de calzadas', 'ancho carril', 'arcen',
       'carriles ascendente', 'carriles descendente', 'limite velocidad',
       'sentidos', 'acera', 'ancho acera', 'anchura calzada',
       'acondicionamiento de interseccion', 'danos via', 'marcas viales',
       'trazado en alzado', 'trazado en planta', 'caracteristica del margen',
       'elementos de separacion', 'elementos de balizamiento',
       'tipo de elementos de tramo', 'circunstancias especiales',
       'delimitacion de la calzada', 'caracteristica funcional de la via',
       'senalizacion del peligro', 'visibilidad de la senalizacion vertical'],
      dtype='object')

In [13]:
delete_vias_columns = ['acera', 'ancho acera', 'anchura calzada', 'acondicionamiento de interseccion',
                       'danos via', 'senalizacion del peligro', 'visibilidad de la senalizacion vertical',
                       'delimitacion de la calzada', 'caracteristica del margen']

drop_columns_df(df_vias, delete_vias_columns);

In [14]:
# Rellenamos con "se desconoce" los valores nulos de las columnas correspondientes
df_vias["numero de calzadas"] = df_vias["numero de calzadas"].fillna("Se desconoce")
df_vias["ancho carril"] = df_vias["ancho carril"].fillna("Se desconoce")
df_vias["arcen"] = df_vias["arcen"].fillna("Se desconoce")
df_vias["trazado en alzado"] = df_vias["trazado en alzado"].fillna("Se desconoce")
df_vias["trazado en planta"] = df_vias["trazado en planta"].fillna("Se desconoce")
df_vias["elementos de balizamiento"] = df_vias["elementos de balizamiento"].fillna("Se desconoce")
df_vias["sentidos"] = df_vias["sentidos"].fillna("Se desconoce")

# Rellenamos los valores nulos del resto de columnas en las que tenemos los datos
df_vias["tipo de elementos de tramo"] = df_vias["tipo de elementos de tramo"].fillna("Ninguno")
df_vias["circunstancias especiales"] = df_vias["circunstancias especiales"].fillna("Ninguna")
df_vias["caracteristica funcional de la via"] = df_vias["caracteristica funcional de la via"].fillna("Ninguna de las anteriores")

df_vias["elementos de separacion"] = df_vias["elementos de separacion"].fillna("Otro")

In [15]:
# A continuación se procede a tratar la columna de "limite de velocidad"
# Imputamos los valores de la media del límiite de velocidad en función del trazado en planta

mask_nan = df_vias['limite velocidad'].isna()

# Iterar sobre los índices de los valores NaN en 'limite velocidad'
for i in df_vias[mask_nan].index:
    
    trazado = df_vias.at[i, 'trazado en planta']
    
    mask_carretera = df_vias['trazado en planta'] == trazado
    
    # Filtrar las filas que coinciden en 'trazado en planta' y que no tengan NaN en 'limite velocidad'
    valores_velocidad = df_vias[mask_carretera]['limite velocidad'].dropna()
    
    # Si hay valores disponibles para calcular la moda
    if not valores_velocidad.empty:
        # Calcular la moda (valor más frecuente)
        moda_limite_velocidad = stats.mode(valores_velocidad)[0]
        
        # Redondear la moda a la decena más cercana
        moda_limite_velocidad_redondeada = redondear_decena(moda_limite_velocidad)
        
        # Asignar el valor redondeado a la fila actual con NaN en 'limite velocidad'
        df_vias.at[i, 'limite velocidad'] = moda_limite_velocidad_redondeada

In [16]:
df_vias.isnull().sum()

codigo del accidente                  0
numero de calzadas                    0
ancho carril                          0
arcen                                 0
carriles ascendente                   0
carriles descendente                  0
limite velocidad                      0
sentidos                              0
marcas viales                         0
trazado en alzado                     0
trazado en planta                     0
elementos de separacion               0
elementos de balizamiento             0
tipo de elementos de tramo            0
circunstancias especiales             0
caracteristica funcional de la via    0
dtype: int64

### Dataframe de características de tramos

In [17]:
df_tramos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4251 entries, 0 to 4250
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   codigo del accidente  4251 non-null   int64  
 1   imd                   4227 non-null   float64
 2   tipo entorno          3554 non-null   object 
 3   tipo calzada          3554 non-null   object 
 4   ancho de arcen        3554 non-null   float64
 5   demarcacion           4251 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 199.4+ KB


In [18]:
delete_tram_columns = ['ancho de arcen']

drop_columns_df(df_tramos, delete_tram_columns);

In [19]:
df_tramos["tipo entorno"] = df_tramos["tipo entorno"].fillna("Se desconoce")
df_tramos["tipo calzada"] = df_tramos["tipo calzada"].fillna("Se desconoce")

In [20]:
df_tramos["imd"].describe()

count     4227.000000
mean      6146.893305
std       5616.447878
min         21.000000
25%       1507.000000
50%       4384.000000
75%       9099.000000
max      27504.000000
Name: imd, dtype: float64

In [21]:
quantile_25 = df_tramos["imd"].quantile(0.25)
n_nulos = df_tramos["imd"].isnull().sum()

quantile_25, n_nulos

(np.float64(1507.0), np.int64(24))

In [22]:
df_tramos["imd"] = df_tramos["imd"].fillna(quantile_25)

In [23]:
# Se define el umbral del IMD para clasificar en función de alta o baja densidad
umbral_intensidad = 2000

df_tramos["rango imd"] = np.where(df_tramos["imd"] >= umbral_intensidad, "Alta intensidad", "Baja intensidad")

In [24]:
df_tramos.isnull().sum()

codigo del accidente    0
imd                     0
tipo entorno            0
tipo calzada            0
demarcacion             0
rango imd               0
dtype: int64

### Dataframe de características de personas implicadas

In [25]:
df_personas.columns

Index(['codigo del accidente', 'identificador', 'presunto responsable',
       'presunto error', 'vehiculo', 'posicion', 'tipo de conductor',
       'posicion en el vehiculo', 'tipo de vehiculo', 'no cotabilizable',
       'lesividad', 'tipo de lesiones', 'sexo', 'edad', 'nacionalidad',
       'poblacion de residencia', 'hospital de atenciones',
       'motivo de desplazamiento', 'desplazamiento previsto',
       'tipo de permiso', 'caracteristica del permiso', 'factor de atencion',
       'signos alcohol', 'prueba de alcoholemia', 'tasa de alcohol 1',
       'tasa de alcohol 2', 'signos drogas', 'droga',
       'kilometros en sentido contrario', 'sin triangulo de presenalizacion',
       'sin luces de emergencia', 'prenda reflectante', 'casco',
       'cinturon de seguridad', 'sistema de retencion infantil',
       'accesorio de seguridad de peaton', 'otros accesorios de seguridad',
       'accion del conductor', 'accion del pasajero', 'accion del peaton',
       'presunta infraccion 

In [26]:
delete_personas_columns = ['tipo de conductor', "no cotabilizable", "tipo de lesiones", "hospital de atenciones",
                           "kilometros en sentido contrario", "prenda reflectante", "poblacion de residencia",
                           "tasa de alcohol 2", "droga", "nacionalidad", "prueba de alcoholemia",
                           "sin triangulo de presenalizacion", 'sin luces de emergencia', 'prenda reflectante',
                           "sistema de retencion infantil", "accesorio de seguridad de peaton", "otros accesorios de seguridad",
                           'accion del conductor', 'accion del pasajero']

drop_columns_df(df_personas, delete_personas_columns);

In [27]:
# Rellenamos los valores nulos para estandarizar los datos de los dataframes

# En el caso de "presunto responsable" se considera que si es pasajero, no es el responsable directo del accidente
df_personas["presunto responsable"] = np.where(
    (df_personas["presunto responsable"].isna()) & (df_personas["posicion"] == "Pasajero"), 
    "NO", 
    df_personas["presunto responsable"]
)

df_personas["presunto responsable"] = df_personas["presunto responsable"].fillna("Se desconoce")

In [28]:
df_personas["presunto error"] = df_personas["presunto error"].fillna("Se desconoce")
df_personas["vehiculo"] = df_personas["vehiculo"].fillna("Se desconoce")
df_personas["posicion en el vehiculo"] = df_personas["posicion en el vehiculo"].fillna("Se desconoce")
df_personas["motivo de desplazamiento"] = df_personas["motivo de desplazamiento"].fillna("Se desconoce")
df_personas["desplazamiento previsto"] = df_personas["desplazamiento previsto"].fillna("Se desconoce")
df_personas["factor de atencion"] = df_personas["factor de atencion"].fillna("Se desconoce")
df_personas["presunta infraccion del conductor"] = df_personas["presunta infraccion del conductor"].fillna("Se desconoce")
df_personas["presunta infraccion de velocidad"] = df_personas["presunta infraccion de velocidad"].fillna("Se desconoce")
df_personas["tipo de permiso"] = df_personas["tipo de permiso"].fillna("Se desconoce")
df_personas["caracteristica del permiso"] = df_personas["caracteristica del permiso"].fillna("Se desconoce")

df_personas["tipo de vehiculo"] = df_personas["tipo de vehiculo"].fillna("Sin especificar")

df_personas["signos alcohol"] = df_personas["signos alcohol"].fillna("Se ignoran")
df_personas["cinturon de seguridad"] = df_personas["cinturon de seguridad"].fillna("Se ignora")
df_personas["casco"] = df_personas["casco"].fillna("Se ignora")

df_personas["tasa de alcohol 1"] = df_personas["tasa de alcohol 1"].fillna(0)

df_personas["signos drogas"] = df_personas["signos drogas"].fillna("NO")

df_personas["accion del peaton"] = df_personas["accion del peaton"].fillna("Peaton no interviene")
df_personas["presunta infraccion del peaton"] = df_personas["presunta infraccion del peaton"].fillna("Peaton no interviene")


In [29]:
df_personas.isnull().sum()

codigo del accidente                   0
identificador                          0
presunto responsable                   0
presunto error                         0
vehiculo                               0
posicion                               0
posicion en el vehiculo                0
tipo de vehiculo                       0
lesividad                              0
sexo                                   0
edad                                 146
motivo de desplazamiento               0
desplazamiento previsto                0
tipo de permiso                        0
caracteristica del permiso             0
factor de atencion                     0
signos alcohol                         0
tasa de alcohol 1                      0
signos drogas                          0
casco                                  0
cinturon de seguridad                  0
accion del peaton                      0
presunta infraccion del conductor      0
presunta infraccion de velocidad       0
presunta infracc

In [30]:
# Se definen las agrupaciones de vehículos en función del tipo de vehículo
bicicletas = ['Bicicleta']

camiones = ['Camión rígido', 'Tractocamión (cabeza tractora)', 'Vehículo articulado']

furgoneta = ["Furgoneta"]

maquinaria_obra_agricola = ['Maquinaria agrícola', 'Maquinaria de obras y servicios']

motocicletas = ['Ciclomotor', 'Motocicleta (Sin determinar cilindrada)', 
                'Motocicleta hasta 125 cc', 'Motocicleta > 125 cc']

turismos = ['Turismo']

vmp = ['Vehículo Movilidad Personal']

otros = ['Todo terreno', 'Sin especificar','Cuadriciclo ligero', 'Quad ligero',
         'Cuadriciclo no ligero', 'Autocaravana', 'Autobús', 'Quad no ligero']

In [31]:
# Se crean nuevas columnas con valores booleanos para definir el tipo de vehículo
tipos_vehiculo = {
    'bicicleta': bicicletas,
    'camion': camiones,
    'furgoneta': furgoneta,
    'maquinaria obra agricola': maquinaria_obra_agricola,
    'motocicleta': motocicletas,
    'turismo' : turismos,
    'vmp' : vmp,
    'otro' : otros
}

clasificar_vehiculos(df_personas, tipos_vehiculo, "tipo de vehiculo");

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656 entries, 0 to 7655
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   codigo del accidente               7656 non-null   int64  
 1   identificador                      7656 non-null   object 
 2   presunto responsable               7656 non-null   object 
 3   presunto error                     7656 non-null   object 
 4   vehiculo                           7656 non-null   object 
 5   posicion                           7656 non-null   object 
 6   posicion en el vehiculo            7656 non-null   object 
 7   tipo de vehiculo                   7656 non-null   object 
 8   lesividad                          7656 non-null   object 
 9   sexo                               7656 non-null   object 
 10  edad                               7510 non-null   float64
 11  motivo de desplazamiento           7656 non-null   objec

In [32]:
df_personas = df_personas.drop(columns = "tipo de vehiculo", axis = 1)

In [33]:
#Definimos el rango de la tasa de alcohol de las personas implicadas en los accidentes
df_personas = rango_edad(df_personas, "edad")

In [34]:
#Definimos el rango de edad de las personas implicadas en los accidentes
df_personas = rango_alcohol(df_personas, "tasa de alcohol 1")

### Dataframe de vehículos implicados

In [35]:
df_vehiculos.columns

Index(['codigo del accidente', 'identificador', 'tipo de vehiculo',
       'nacionalidad', 'fecha de matriculacion', 'numero ocupantes',
       'conductor', 'matricula', 'color del vehiculo', 'marca', 'modelo',
       'anomalia previa', 'itv correcta', 'seguro obligatorio',
       'sin conductor', 'remolque', 'tipo remolque', 'mma', 'disco tacografo',
       'leido el disco tacografo', 'airbags disparados', 'velocidad final',
       'mercancias peligrosas', 'codigo onu', 'transporte especial',
       'vehiculo adaptado', 'usa el alumbrado reglamentario',
       'respeta el descanso diario', 'supera la conduccion diaria',
       'supera la conduccion continuada', 'maniobra del vehiculo',
       'sentido de ciruclacion', 'aproximacion al nudo',
       'lugar de ciruclacion', 'posicion respecto a la via',
       'posicion respecto a la calzada', 'posicion respecto al carril',
       'area mas danada del vehiculo', 'fugado', 'vehiculo incendiado'],
      dtype='object')

In [36]:
delete_vehiculos_columns = ["nacionalidad", "conductor", "matricula", "color del vehiculo", "modelo", "seguro obligatorio",
                            "sin conductor", "tipo remolque", "mma", 'disco tacografo','leido el disco tacografo',
                            "velocidad final", "codigo onu", 'usa el alumbrado reglamentario', 'maniobra del vehiculo',
                            "supera la conduccion diaria", "supera la conduccion continuada", "respeta el descanso diario",
                            "fugado", "vehiculo incendiado"]

drop_columns_df(df_vehiculos, delete_vehiculos_columns);

In [37]:
df_vehiculos.isnull().sum()

codigo del accidente                 0
identificador                        0
tipo de vehiculo                     0
fecha de matriculacion             414
numero ocupantes                     0
marca                              373
anomalia previa                      0
itv correcta                       419
remolque                            21
airbags disparados                4947
mercancias peligrosas               19
transporte especial                 19
vehiculo adaptado                   19
sentido de ciruclacion              19
aproximacion al nudo                21
lugar de ciruclacion               103
posicion respecto a la via        2300
posicion respecto a la calzada      19
posicion respecto al carril         19
area mas danada del vehiculo        19
dtype: int64

In [38]:
lista_se_desconoce = ["marca", 'airbags disparados', 'sentido de ciruclacion', 'aproximacion al nudo',
                      'lugar de ciruclacion', 'posicion respecto a la via', 'posicion respecto a la calzada',
                      'posicion respecto al carril', 'area mas danada del vehiculo']


replace_nan(df_vehiculos, lista_se_desconoce, "Se desconoce");

In [39]:
lista_no = ["itv correcta", "remolque", "mercancias peligrosas", "transporte especial",
            'vehiculo adaptado']

replace_nan(df_vehiculos, lista_no, "NO");

In [40]:
# Se definen las agrupaciones de vehículos en función del tipo de vehículo
bicicletas = ['Bicicleta']

camiones = ['Camión rígido', 'Tractocamión (cabeza tractora)', 'Vehículo articulado']

furgoneta = ["Furgoneta"]

maquinaria_obra_agricola = ['Maquinaria agrícola', 'Maquinaria de obras y servicios']

motocicletas = ['Ciclomotor', 'Motocicleta (Sin determinar cilindrada)', 
                'Motocicleta hasta 125 cc', 'Motocicleta > 125 cc']

turismos = ['Turismo']

vmp = ['Vehículo Movilidad Personal']

otros = ['Todo terreno', 'Sin especificar','Cuadriciclo ligero', 'Quad ligero',
         'Cuadriciclo no ligero', 'Autocaravana', 'Autobús', 'Quad no ligero']

In [41]:
# Se crean nuevas columnas con valores booleanos para definir el tipo de vehículo
tipos_vehiculo = {
    'bicicleta': bicicletas,
    'camion': camiones,
    'furgoneta': furgoneta,
    'maquinaria obra agricola': maquinaria_obra_agricola,
    'motocicleta': motocicletas,
    'turismo' : turismos,
    'vmp' : vmp,
    'otro' : otros
}

clasificar_vehiculos(df_vehiculos, tipos_vehiculo, "tipo de vehiculo");

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5763 entries, 0 to 5762
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   codigo del accidente            5763 non-null   int64         
 1   identificador                   5763 non-null   object        
 2   tipo de vehiculo                5763 non-null   object        
 3   fecha de matriculacion          5349 non-null   datetime64[ns]
 4   numero ocupantes                5763 non-null   int64         
 5   marca                           5763 non-null   object        
 6   anomalia previa                 5763 non-null   object        
 7   itv correcta                    5763 non-null   object        
 8   remolque                        5763 non-null   object        
 9   airbags disparados              5763 non-null   object        
 10  mercancias peligrosas           5763 non-null   object        
 11  tran

In [42]:
df_vehiculos = df_vehiculos.drop(columns = "tipo de vehiculo")

## Guardado de datos en Excel

In [43]:
ruta_guardado = r'C:\Users\alejandro.gc\Desktop\Proyectos\Accidentalidad\Accidentes_VALENCIA\data\CLEAN_DATA\clean_data.xlsx'

ruta_archivo = os.path.join(ruta_guardado)

with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:
    df_accidentes.to_excel(writer, sheet_name='Accidentes', index=False)
    df_vias.to_excel(writer, sheet_name='Vias', index=False)
    df_tramos.to_excel(writer, sheet_name='Tramos', index=False)
    df_personas.to_excel(writer, sheet_name='Personas', index=False)
    df_vehiculos.to_excel(writer, sheet_name='Vehiculos', index=False)